In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("Resources/plays.csv")
df.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False


In [57]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df = df.drop(columns=['gameClock'])
df.head()

,gameId,playId,quarter,down,yardsToGo,yardlineNumber,defendersInTheBox,numberOfPassRushers,preSnapVisitorScore,preSnapHomeScore,absoluteYardlineNumber,passResult,offensePlayResult,playResult,epa,isDefensivePI
29,2018090600,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,C,4,19,1.299050,False
45,2018090600,2503,3,2,10,32,5.0,4.0,6.0,3.0,42.0,I,0,-12,-1.868122,False
91,2018090901,439,1,3,9,43,6.0,4.0,0.0,0.0,53.0,C,0,0,-1.770704,False
92,2018090901,493,1,1,10,14,7.0,4.0,0.0,0.0,96.0,S,-5,10,1.019909,False
108,2018090901,1452,2,3,6,33,5.0,5.0,7.0,0.0,77.0,C,1,-14,-2.734050,False


In [58]:
# Remove Space for `FALSE POSITIVE` category
mask = df["passResult"] == "FALSE POSITIVE"
df.loc[mask, "passResult"] = "False_Positive"
df["passResult"]

29       C
45       I
91       C
92       S
108      C
        ..
18473    C
18483    C
18519    C
18554    I
18564    C
Name: passResult, Length: 565, dtype: object

In [59]:
from sklearn.model_selection import train_test_split
y = df["passResult"]
X = df.drop(columns=["passResult"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [60]:
X_train.head()

,gameId,playId,quarter,down,yardsToGo,yardlineNumber,defendersInTheBox,numberOfPassRushers,preSnapVisitorScore,preSnapHomeScore,absoluteYardlineNumber,offensePlayResult,playResult,epa,isDefensivePI
2264,2018092000,1674,2,3,12,35,6.0,5.0,14.0,0.0,45.0,-6,-6,-1.649617,False
167,2018090901,5211,5,1,10,32,5.0,5.0,21.0,21.0,42.0,0,0,-0.241342,False
1019,2018091300,1865,2,3,9,38,5.0,4.0,7.0,21.0,48.0,22,22,2.617156,False
8135,2018102805,2317,3,3,10,39,6.0,4.0,14.0,16.0,49.0,18,29,2.789296,False
13003,2018120200,1199,2,2,7,22,7.0,4.0,7.0,3.0,88.0,-4,-78,-10.951801,False


In [61]:


# from sklearn.preprocessing import MinMaxScaler
# X_scaler = MinMaxScaler().fit(X_train)

# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [62]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

In [65]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [66]:
y_train_categorical.shape

(423, 4)

In [67]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [68]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [69]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________
